# Neural bug detectors
This accompanying notebook is an interactive demo for the neural bug detectors used in our study. The goal is to localize variable misuse bugs in Java code.

## Graph Neural Network

In [1]:
# Load the pre-trained GNN checkpoint
from nbfbaselines import NBFModel

gnn_detector = NBFModel.from_pretrained("gnn-study-java")

Now, we are ready for the first test. Let's start with a simple example:

In [2]:
# To run the GNN, we simply call the object with the Java code for the analysis

gnn_detector("""

public int add(int x, int y){
    return x + x;
}

""")

[{'text': 'public int add ( int x , int y ) {\n     return x + y ;\n }',
  'before': 'x',
  'after': 'y',
  'prob': 1.0}]

If you run the cell above, you will see that the GNN successfully detected that we likely want to use `y` for the addition. It is also very confident that adding `x` with itself unlikely to happen. 

# Transformer
We try now the same for the Transformer bug detector

In [3]:
# Load the pre-trained Transformer checkpoint
from nbfbaselines import NBFModel

transformer_detector = NBFModel.from_pretrained("transformer-study-java")

Now, we can run the transformer in a similar way to the GNN

In [4]:
# To run the Transformer, we simply call the object with the Java code for the analysis

transformer_detector("""

public int add(int x, int y){
    return x + x;
}

""")

[{'text': 'public int add ( int x , int y ) {\n     return x + y ;\n }',
  'before': 'x',
  'after': 'y',
  'prob': 0.9997006352337359}]

We see that the transformer detects also the same bug.

## Detecting fixes with code_diff

In [5]:
import code_diff

detector = gnn_detector

buggy_code = """

public int add(int x, int y){
    return x + x;
}

"""

fixed_code = detector(buggy_code)[0]["text"]

diff = code_diff.difference(buggy_code, fixed_code, lang = "java") # x -> y
changed_line = diff.source_ast.position[0][0] # 3

print("Minimal change: %s" % str(diff))
print("Changed line: %d" % changed_line)

Minimal change: x -> y
Changed line: 3
